In [1]:
import pandas as pd
from datetime import datetime
import os
import jsons
import numpy as np
jsons.suppress_warnings()

In [2]:
articleFilename = "./API ratification territorial application reservations - With UK updated-Jul23.xlsx"
courtFilename = "./Countries_and_national_courts.csv"
# path = "D:/Projects/robolawyer/applicationForm/dataPreparation/countryAPI"
path = "/home/aman/Projects/robolawyer/applicationForm/dataPreparation/countryAPI"
os.chdir(path)
final_list = []
country_json = {}
article_json = {}
country_json = {}
court_json = {}
fill_dt = datetime(2100,1,1)
fill_na = "N/A"

In [3]:
sheet_name = [i for i in range(1, 46)]
data = pd.read_excel(articleFilename, sheet_name=sheet_name, index_col=0)
court = pd.read_csv(courtFilename, header=0, sep=",")



for element in data:
    data[element]['Date'] = data[element]['Date'].fillna(fill_dt)
    data[element]['Active/not active'] = data[element]['Active/not active'].fillna(fill_na)
    data[element]['Reservations/Derogations/Declarations'] = data[element]['Reservations/Derogations/Declarations'].fillna(fill_na)
    data[element]['Article'] = data[element]['Article'].fillna(fill_na)
    data[element]['Full text'] = data[element]['Full text'].fillna(fill_na)
    


In [4]:
def getFormattedArticleText(text):
    text = text.split('<br/>\n')
    if len(text) == 1:
        return [{'mainText': text,'points': []}]
    res = []
    flag = 0
    temp = {'mainText': '','points': []}
    for i in text:
        if i[0] != '(':
            # main text
            if flag == 1 or flag == 2:
                res.append(temp)
                temp = {'mainText': '','points': []}
            flag = 1
            temp['mainText'] = i
            
        else:
            # points
            flag = 2
            temp['points'].append(i)
    if flag == 2 or flag == 1:
        res.append(temp)
    return res
    

In [7]:
str = "1. Everyone has the right to liberty and security of person. No one shall be deprived of his liberty save in the following cases and in accordance with a procedure prescribed by law:<br/>\n(a) the lawful detention of a person after conviction by a competent court;<br/>\n(b) the lawful arrest or detention of a person for noncompliance with the lawful order of a court or in order to secure the fulfilment of any obligation prescribed by law;<br/>\n(c) the lawful arrest or detention of a person effected for the purpose of bringing him before the competent legal authority on reasonable suspicion of having committed an offence or when it is reasonably considered necessary to prevent his committing an offence or fleeing after having done so;<br/>\n(d) the detention of a minor by lawful order for the purpose of educational supervision or his lawful detention for the purpose of bringing him before the competent legal authority;<br/>\n(e) the lawful detention of persons for the prevention of the spreading of infectious diseases, of persons of unsound mind, alcoholics or drug addicts or vagrants;<br/>\n(f) the lawful arrest or detention of a person to prevent his effecting an unauthorised entry into the country or of a person against whom action is being taken with a view to deportation or extradition.<br/>\n2. Everyone who is arrested shall be informed promptly, in a language which he understands, of the reasons for his arrest and of any charge against him.<br/>\n3. Everyone arrested or detained in accordance with the provisions of paragraph 1 (c) of this Article shall be brought promptly before a judge or other officer authorised by law to exercise judicial power and shall be entitled to trial within a reasonable time or to release pending trial. Release may be conditioned by guarantees to appear for trial.<br/>\n4. Everyone who is deprived of his liberty by arrest or detention shall be entitled to take proceedings by which the lawfulness of his detention shall be decided speedily by a court and his release ordered if the detention is not lawful.<br/>\n5. Everyone who has been the victim of arrest or detention in contravention of the provisions of this Article shall have an enforceable right to compensation."
# getFormattedArticleText(str)


In [5]:
import json
textArray = data[1]['Full text']
print(repr(textArray[6]))
print("\n\n")
print(repr(textArray[1]))
resultList = []
for text in textArray:
    resultList.append(getFormattedArticleText(text))

# resultList

'1. No one shall be held guilty of any criminal offence on account of any act or omission which did not constitute a criminal offence under national or international law at the time when it was committed. Nor shall a heavier penalty be imposed than the one that was applicable at the time the criminal offence was committed. <br/>\n2. This Article shall not prejudice the trial and punishment of any person for any act or omission which, at the time when it was committed, was criminal according to the general principles of law recognised by civilised nations.'



'1. Everyone’s right to life shall be protected by law. No one shall be deprived of his life intentionally save in the execution of a sentence of a court following his conviction of a crime for which this penalty is provided by law. <br/>\n2. Deprivation of life shall not be regarded as inflicted in contravention of this Article when it results from the use of force which is no more than absolutely necessary: <br/>\n(a) in defence

In [6]:
for item in range(1, len(data)):
    country_json[data[item]['Country'][1]] = {}
    
court_country = court['Country']

In [8]:
for item in country_json:
    for record in data:
        if item == data[record]['Country'].any():
            country_json[item]['Date'] = pd.to_datetime(data[record]['Date']).dt.strftime("%d-%m-%Y")
            country_json[item]['Active'] = data[record]['Active/not active']
            country_json[item]['Reservations'] = data[record]['Reservations/Derogations/Declarations']
            country_json[item]['ratDate'] = pd.to_datetime(data[record]['Date'].min()).strftime("%d-%m-%Y")
        if item == data[record]['Country'].all():
            print("hello")
            article_json['Article'] = data[record]['Article'] 
            article_json['Full text'] = resultList
        
        if item in court['Country'].tolist():
            myIndex = court_country[court_country==item].index[0]
            if type(court['ProceedingType3'].iloc[myIndex])==float and type(court['ProceedingType2'].iloc[myIndex])==float:
                country_json[item]["Court"] = {'ProceedingType1':str(court['ProceedingType1'].iloc[myIndex]),
                                               'Court1': str(court['Court1'].iloc[myIndex])}

            elif type(court['ProceedingType3'].iloc[myIndex])==float and type(court['ProceedingType2'].iloc[myIndex])==str:

                country_json[item]["Court"] = {'ProceedingType1':str(court['ProceedingType1'].iloc[myIndex]),
                                               'Court1': str(court['Court1'].iloc[myIndex]),
                                          'ProceedingType2':str(court['ProceedingType2'].iloc[myIndex]),
                                               'Court2': str(court['Court2'].iloc[myIndex])}
            else:
                country_json[item]["Court"] = {'ProceedingType1': str(court['ProceedingType1'].iloc[myIndex]), 
                                               'Court1': str(court['Court1'].iloc[myIndex]),
                                          'ProceedingType2': str(court['ProceedingType2'].iloc[myIndex]),
                                               'Court2': str(court['Court2'].iloc[myIndex]),
                                          'ProceedingType3': str(court['ProceedingType3'].iloc[myIndex]),
                                               'Court3': str(court['Court3'].iloc[myIndex])}



In [57]:
with open('final.json', 'w', encoding = "UTF-8") as ps:
    ps.write(jsons.dumps(country_json))

In [ ]:
# with open('article.json', 'w', encoding="UTF-8") as ps:
#     ps.write(jsons.dumps(article_json))